<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Рекуррентная нейронная сеть и </b> <span style="font-weight:bold; color:green">TensorFlow</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_hse@mail.ru)</span></div>

<p>Подключение стилей оформления</p>

In [ ]:
%%html
<link href="css/style.css" rel="stylesheet" type="text/css">

In [ ]:
import inspect
import time

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from lib import rnn_lang_modeling_reader as reader

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Загрузка исходных данных</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p><b>Вариант 1.</b> Из командной строки</p>

In [ ]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz

In [ ]:
!tar xvf simple-examples.tgz

<p><b>Вариант 2.</b> Средствами Python</p>

In [ ]:
import urllib.request
import shutil
import os
import tarfile

In [ ]:
url = "http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz"

filename = "data/rnn-lang-modeling/rnn-simple.tgz"

os.makedirs(os.path.dirname(filename), exist_ok=True)

# Загрузка архива
with urllib.request.urlopen(url) as response:
    with open(filename, 'wb') as output:
        shutil.copyfileobj(response, output)

# Распаковка
with tarfile.open(filename) as tar:
    tar.extractall(path="data/rnn-lang-modeling/")
    

<p>Директория с исходными данными и для записи логов и модели</p>

In [ ]:
data_path = "data/rnn-lang-modeling/simple-examples/data"
save_path = "log/rnn-lang-modeling/log"

<p><i>Загрузка данных и преобразование в вектор индексов слов</i></p>

In [ ]:
train_data, valid_data, test_data, vocabulary = reader.ptb_raw_data(data_path)

In [ ]:
train_data[:10]

<p><i>Количество слов (токенов) в обучающем подмножестве</i></p>

In [ ]:
len(train_data)

<p><i>Индекс первого слова в тестовом подмножестве</i></p>

In [ ]:
test_data[0]

<p><i>Словарь преобразования слов в индексы</i></p>

In [ ]:
vocabulary

In [ ]:
len(vocabulary)

<p><i>Обратное пребразование индекса первого слова тестового подмножества</i></p>

In [ ]:
for key in vocabulary:
    if vocabulary[key] == test_data[0]:
        print(key)

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Этапы построения сети</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Параметры модели</p>

<p><i>Структура сети</i></p>

In [ ]:
num_steps = 24 # количество развертки LSTM
hidden_size = 200 # количество LSTM единиц
vocab_size = 10000 # размер словаря

<p><i>Параметры инициализации весов и параметры обучения</i></p>

In [ ]:
init_scale = 0.1 # the initial scale of the weights
learning_rate = 1.0 # коэффициент обучения
max_grad_norm = 5 # the maximum permissible norm of the gradient
max_epoch = 4 # the number of epochs trained with the initial learning rate
max_max_epoch = 13 # количество эпох
keep_prob = 1.0 # вероятность сохранения веса в dropout слое
lr_decay = 0.5 # the decay of the learning rate for each epoch after "max_epoch"
batch_size = 20 # размер batch

<p>Построение сети</p>

<p><i>Входные данные</i></p>

<p><i>Формирование исходных данных</i></p>

In [ ]:
x_train, y_train = reader.ptb_producer(train_data, batch_size, num_steps, name=None)
x_train

In [ ]:
y_train

<p><i>Преобразование слов в распределенное представление (word2vec)</i></p>

In [ ]:
embedding = tf.get_variable("embedding", [vocab_size, hidden_size], dtype=tf.float32)
embedding

In [ ]:
inputs = tf.nn.embedding_lookup(embedding, x_train)
inputs

<p><i>Создание двух слоев LSTM</i></p>

In [ ]:
lstm_layer_1 = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0, 
                                            state_is_tuple=True,
                                            reuse=tf.get_variable_scope().reuse)

In [ ]:
lstm_layer_2 = tf.contrib.rnn.BasicLSTMCell(hidden_size, forget_bias=0.0, 
                                            state_is_tuple=True, 
                                            reuse=tf.get_variable_scope().reuse)

In [ ]:
lstm_layer_2

<p><i>Объединение слоев в одну структуру</i></p>

In [ ]:
multiple_cell = tf.contrib.rnn.MultiRNNCell([lstm_layer_1, lstm_layer_2], state_is_tuple=True)

<p><i>Инициализация начальных значений состояний</i></p>

In [ ]:
state = multiple_cell.zero_state(batch_size, tf.float32)
state

<p><i>Определение начальных и выходных состояний</i></p>

In [ ]:
inputs = tf.unstack(inputs, num=num_steps, axis=1)
inputs

In [ ]:
outputs, state = tf.contrib.rnn.static_rnn(multiple_cell, inputs, initial_state=state)

In [ ]:
len(outputs)

In [ ]:
state[0][1]

In [ ]:
#outputs = []
#with tf.variable_scope("RNN"):
#    for time_step in range(num_steps):
        
        #if time_step > 0: 
            # 
        #    tf.get_variable_scope().reuse_variables()
            
#        cell_output, state = multiple_cell(inputs[:, time_step, :], state)
#        outputs.append(cell_output)

<p><i>Вычисление вероятности появления данных из y_train</i></p>

In [ ]:
output = tf.reshape(tf.stack(axis=1, values=outputs), [-1, hidden_size])
output

In [ ]:
24*20

In [ ]:
softmax_w = tf.get_variable("softmax_w", [hidden_size, vocab_size], dtype=tf.float32)
softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)

In [ ]:
logits = tf.matmul(output, softmax_w) + softmax_b

In [ ]:
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits],
                                                          [tf.reshape(y_train, [-1])], 
                                                          [tf.ones([batch_size * num_steps], dtype=tf.float32)])
loss

<p><i>Функция потерь</i></p>

In [ ]:
cost = tf.reduce_sum(loss) / batch_size

<p><i>Конечное состояние</i></p>

In [ ]:
final_state = state

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Запуск обучения</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<p>Параметры сети</p>

In [ ]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1 # the initial scale of the weights
    learning_rate = 1.0 # the initial value of the learning rate
    max_grad_norm = 5 # the maximum permissible norm of the gradient
    num_layers = 2 # the number of LSTM layers
    num_steps = 20 # the number of unrolled steps of LSTM
    hidden_size = 200 # the number of LSTM units
    max_epoch = 4 # the number of epochs trained with the initial learning rate
    max_max_epoch = 13 # the total number of epochs for training
    keep_prob = 1.0 # the probability of keeping weights in the dropout layer
    lr_decay = 0.5 # the decay of the learning rate for each epoch after "max_epoch"
    batch_size = 20 # the batch size
    vocab_size = 10000 # the vocabulary size

class TestConfig(object):
    """Tiny config, for testing."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch = 1
    max_max_epoch = 1
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000

<p>Входные данные</p>

In [ ]:
class PTBInput(object):
    """The input data."""

    def __init__(self, config, data, name=None):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
        self.input_data, self.targets = reader.ptb_producer(data, batch_size, num_steps, name=name)

<p>Модель сети</p>

In [ ]:
class PTBModel(object):
    
    """The PTB model."""

    def __init__(self, is_training, config, input_):
        
        self._input = input_
        
        batch_size = input_.batch_size
        num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size

        # Slightly better results can be obtained with forget gate biases
        # initialized to 1 but the hyperparameters of the model would need to be
        # different than reported in the paper.
        
        def lstm_cell():
            return tf.contrib.rnn.BasicLSTMCell(size, 
                                                forget_bias=0.0, 
                                                state_is_tuple=True,
                                                reuse=tf.get_variable_scope().reuse)
            
        attn_cell = lstm_cell
        
        
        if is_training and config.keep_prob < 1:
            def attn_cell():
                return tf.contrib.rnn.DropoutWrapper(lstm_cell(), output_keep_prob=config.keep_prob)
        
        cell = tf.contrib.rnn.MultiRNNCell([attn_cell() for _ in range(config.num_layers)], 
                                           state_is_tuple=True)

        self._initial_state = cell.zero_state(batch_size, tf.float32)
        
        with tf.device("/cpu:0"):
            embedding = tf.get_variable("embedding", [vocab_size, size], dtype=tf.float32)
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)

        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)

        # Simplified version of models/tutorials/rnn/rnn.py's rnn().
        # This builds an unrolled LSTM for tutorial purposes only.
        # In general, use the rnn() or state_saving_rnn() from rnn.py.
        #
        # The alternative version of the code below is:
        #
        # inputs = tf.unstack(inputs, num=num_steps, axis=1)
        # outputs, state = tf.contrib.rnn.static_rnn(
        #     cell, inputs, initial_state=self._initial_state)
        
        outputs = []
        state = self._initial_state
        with tf.variable_scope("RNN"):
            for time_step in range(num_steps):
                if time_step > 0: 
                    tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)

        output = tf.reshape(tf.stack(axis=1, values=outputs), [-1, size])
        softmax_w = tf.get_variable("softmax_w", 
                                    [size, vocab_size], 
                                    dtype=tf.float32)
        softmax_b = tf.get_variable("softmax_b", 
                                    [vocab_size], 
                                    dtype=tf.float32)
        logits = tf.matmul(output, softmax_w) + softmax_b
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits],
                                                                  [tf.reshape(input_.targets, [-1])],
                                                                  [tf.ones([batch_size * num_steps], 
                                                                           dtype=tf.float32)])
        self._cost = cost = tf.reduce_sum(loss) / batch_size
        self._final_state = state

        if not is_training:
            return

        self._lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                          config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars),
                                                   global_step=tf.contrib.framework.get_or_create_global_step())

        self._new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
        self._lr_update = tf.assign(self._lr, self._new_lr)

    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

    @property
    def input(self):
        return self._input

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr

    @property
    def train_op(self):
        return self._train_op

<p>Запуск эпохи</p>

In [ ]:
def run_epoch(session, model, eval_op=None, verbose=False):
    """Runs the model on the given data."""
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)

    fetches = {
      "cost": model.cost,
      "final_state": model.final_state,
    }
    if eval_op is not None:
        fetches["eval_op"] = eval_op

    for step in range(model.input.epoch_size):
        feed_dict = {}
        
        #cell state and hidden state
        for i, (c, h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h

            
        # i = input_gate, j = new_input, f = forget_gate, o = output_gate
        # concat = _conv_linear([inputs, h], self.filter_size, self.num_features * 4, True)
        # i, j, f, o = tf.split(axis=3, num_or_size_splits=4, value=concat)
            
        
        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]

        costs += cost
        iters += model.input.num_steps

        if verbose and step % (model.input.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
                 iters * model.input.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

<p>Конфигурирование</p>

In [ ]:
config = SmallConfig()
config.max_max_epoch=13

eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

<p>Запуск обучения</p>

In [ ]:
with tf.Graph().as_default():
    
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
    
    with tf.name_scope("Train"):
        
        train_input = PTBInput(config=config, data=train_data, name="TrainInput")
        
        with tf.variable_scope("Model", reuse=None, initializer=initializer):
            m = PTBModel(is_training=True, config=config, input_=train_input)
        
        tf.summary.scalar("Training_Loss", m.cost)
        tf.summary.scalar("Learning_Rate", m.lr)
        
    with tf.name_scope("Valid"):
        
        valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
        
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
        
        tf.summary.scalar("Validation_Loss", mvalid.cost)
        
    with tf.name_scope("Test"):
        
        test_input = PTBInput(config=eval_config, data=test_data, name="TestInput")
        
        with tf.variable_scope("Model", reuse=True, initializer=initializer):
            mtest = PTBModel(is_training=False, config=eval_config, input_=test_input)

    sv = tf.train.Supervisor(logdir=save_path)
    
    with sv.managed_session() as session:
        for i in range(config.max_max_epoch):
            lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
            m.assign_lr(session, config.learning_rate * lr_decay)
            print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
            train_perplexity = run_epoch(session, m, eval_op=m.train_op,
                                     verbose=True)
            print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
            valid_perplexity = run_epoch(session, mvalid)
            print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
        
        test_perplexity = run_epoch(session, mtest)
        print("Test Perplexity: %.3f" % test_perplexity)
        
        
        if save_path:
            print("Saving model to %s." % save_path)
            sv.saver.save(session, save_path, global_step=sv.global_step)

<p>Восстановление модели</p>

In [ ]:
.meta - граф
.data - значения весов, переменных


<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

In [ ]:
https://github.com/tensorflow/models/tree/master/tutorials/rnn